# NLP to SQL

Answer to Natural Language question, such as "What counties had the top sales?" and convert that into a SQL query. We will then use that SQL to query the data (which in this example will come from a .csv file we read in with Pandas and set-up a temporary DB in RAM) and then report back the results!

## Imports

In [14]:
import os
import pandas as pd
import openai


## Data


Data will be some example Sales Data from:  https://www.kaggle.com/datasets/kyanyoga/sample-sales-data


In [3]:
df = pd.read_csv("sales_data_sample.csv")

In [4]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Querying Data

In [5]:
Pandas Query for Sum of Sales by Quarter 
df.groupby("QTR_ID").sum()['SALES']

C:\Users\Soumyansh\AppData\Local\Temp\ipykernel_1164\1598294123.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("QTR_ID").sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64


Query via SQL Syntax, Setting-up a temporary in memory (RAM) database, basically export this CSV-->Pandas DF--> SQL DB

## SQL Database Set-up

In [6]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [7]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

Here we push our entire DataFrame to a table called Sales:

In [8]:
data = df.to_sql(name='Sales',con=temp_db)


2023-04-11 12:18:13,030 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-04-11 12:18:13,031 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 12:18:13,033 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-04-11 12:18:13,035 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 12:18:13,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-11 12:18:13,047 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20



### Connecting to SQL Database:

Using SQL Alchemy to establish a connection to this temporary database and query it for the results:

In [9]:
with temp_db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

2023-04-11 12:18:17,720 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2023-04-11 12:18:17,720 INFO sqlalchemy.engine.Engine [generated in 0.00203s] ()


In [10]:
result.all()

[(10407, 14082.8)]

## OpenAI API

### Set-up Open AI API Key

Goto account section for the same

In [16]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Inform GPT about the SQL Table Structure

In [12]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """

    prompt = '''### sqlite SQL table, with its properties:
#
# Sales({})
#
'''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [13]:
print(create_table_definition_prompt(df))

### sqlite SQL table, with its properties:
#
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#



### Get Natural Language Request:

Creating a function that grabs the natural language information request:

In [20]:
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [21]:
nlp_text = prompt_input()

Enter information you want to obtain:  Sum of SALES per POSTALCODE


Now let's combine the results in one function:

In [22]:
nlp_text

'Sum of SALES per POSTALCODE'

In [23]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string
    

In [24]:
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n#\n# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n#\n### A query to answer: Sum of SALES per POSTALCODE\nSELECT'

GPT will complete the text above, thus we start to notify it to begin a SQL query by writing "\nSELECT"

Now let's get the response:

### OpenAI API Call

Let's use the Text DaVinci model

In [25]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=combine_prompts(df, nlp_text),
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)


In [26]:
response

<OpenAIObject text_completion id=cmpl-6mPJ8MSAJqRvhV3VtS6d7hti95muS at 0x1ae13fffe00> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE"
    }
  ],
  "created": 1676995046,
  "id": "cmpl-6mPJ8MSAJqRvhV3VtS6d7hti95muS",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 156,
    "total_tokens": 175
  }
}

Building a function to parse the section of the response we want: 

In [27]:
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "Select"+ query
    return query

In [28]:
handle_response(response)

'Select POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

Now we just pass that into our Database:

In [29]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-02-21 09:57:28,355 INFO sqlalchemy.engine.Engine Select POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
2023-02-21 09:57:28,358 INFO sqlalchemy.engine.Engine [generated in 0.00272s] ()


In [30]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287